In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import helper

import json
import os
import sys

In [2]:
course_schedule = []

IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20


RETRY_INTERVAL = 65

In [3]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element("xpath",xpath)

In [4]:
def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [5]:
def create_new_driver():
    #start web driver

    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service

    service = Service(executable_path=r'/usr/bin/chromedriver')

    #setup option for chrome profile
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=service, options=options)

    driver.implicitly_wait(IMPLICIT_WAIT_TIME)
    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    

    return driver, wait


In [6]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element("id",'ctl00_cphContents_txtUsername')
    password = driver.find_element("id",'ctl00_cphContents_txtPassword')
    
    send_keys(username, "DN20092360")
    send_keys(password, "jj1216")
#     username.send_keys("DN23035242")
#     password.send_keys("NTW3BSFH")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))

    sign_in_btn = driver.find_element(By.CLASS_NAME,"submit-button")
    sign_in_btn.click()

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class():
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), '預定課')]")
#     print("click_btn: ", click_btn)
    click_btn.click()
    
def click_book_this_class_now(driver, wait):
    
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), '現在就預定此課')]")

    if click_btn:
        click_btn.click()
        return click_btn

In [7]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [8]:
temp = []

# load schedule from file

# with open("./class_schedules/30 十月 - 05 十一月 2023.json", "r") as read_file:
#     course_schedule = json.load(read_file)

with open("./class_schedules/06 十一月 - 12 十一月 2023.json", "r") as read_file:
    course_schedule = json.load(read_file)

Day_string = "10"

c = helper.search_courses_by_date(Day_string, course_schedule)
# c = helper.search_courses_by_name('Aeri', c)
c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('pm', c)

temp.extend(c)

courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 10 11月 周五
-----------------------

Name: Yoga Tone 2
Teacher: Una
Time: 02:00 - 03:00 下午


Name: Yoga Wheel 瑜珈輪
Teacher: Una
Time: 03:30 - 04:30 下午


Name: Hatha Basic 哈達基礎
Teacher: Una
Time: 12:45 - 01:45 下午




In [9]:
master_counter = 1

In [10]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [11]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [12]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [13]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [14]:
def reserve_class(driver, wait, course_to_be_booked, idx):
    
    global master_counter
    global class_idx
    global num_classes

    stride =RETRY_INTERVAL / 60 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    

    driver.get(book_url)

    click_btn=click_book_this_class_now(driver, wait)
    if click_btn is not None:
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        num_classes = num_classes - 1
    else:
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=RETRY_INTERVAL)
        log_str = "'{}','{}' by '{}',FULL,'{}'\nnext_book_datetime={}\nclick_btn={}\n".format(master_counter, class_name, teacher, datetime.datetime.now(), next_book_datetime, click_btn)
        print(log_str)
        
        if 0==0: #(master_counter - class_idx) % stride == 0:
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
       
                
        driver.close()
        
        driver, wait = create_new_driver()

        sign_in_once(driver, wait)
        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60+(idx%len(course_to_be_booked))*10 )
        print("now={}".format(datetime.datetime.now()))
        print("next_book_datetime={}".format(next_book_datetime))
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked, idx])
    
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [15]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(csv_file_name, "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    
    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60+(idx%len(course_to_be_booked))*10 )
    print("now={}".format(datetime.datetime.now()))
    print("next_book_datetime={}".format(next_book_datetime))
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked, idx])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

now=2023-11-07 13:07:21.933226
next_book_datetime=2023-11-07 13:08:21.933204
now=2023-11-07 13:07:26.649176
next_book_datetime=2023-11-07 13:08:36.649148
now=2023-11-07 13:07:31.619561
next_book_datetime=2023-11-07 13:08:51.619537
master counter = 1, stride = 48.75, class_idx = 1
'1','Yoga Tone 2' by 'Una',FULL,'2023-11-07 13:08:42.574473'
next_book_datetime=2023-11-07 13:09:47.574455
click_btn=None

now=2023-11-07 13:08:47.746303
next_book_datetime=2023-11-07 13:09:47.746280
master counter = 2, stride = 48.75, class_idx = 2
'2','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 13:08:57.342539'
next_book_datetime=2023-11-07 13:10:02.342511
click_btn=None

now=2023-11-07 13:09:02.196964
next_book_datetime=2023-11-07 13:10:12.196936
master counter = 3, stride = 48.75, class_idx = 3
'3','Hatha Basic 哈達基礎' by 'Una',FULL,'2023-11-07 13:09:12.289998'
next_book_datetime=2023-11-07 13:10:17.289968
click_btn=None

now=2023-11-07 13:09:17.108138
next_book_datetime=2023-11-07 13:10:37.108114
master coun

now=2023-11-07 13:24:58.213590
next_book_datetime=2023-11-07 13:26:08.213560
master counter = 33, stride = 48.75, class_idx = 3
'33','Hatha Basic 哈達基礎' by 'Una',FULL,'2023-11-07 13:25:04.080220'
next_book_datetime=2023-11-07 13:26:09.080193
click_btn=None

now=2023-11-07 13:25:08.766897
next_book_datetime=2023-11-07 13:26:28.766873
master counter = 34, stride = 48.75, class_idx = 1
'34','Yoga Tone 2' by 'Una',FULL,'2023-11-07 13:25:52.878624'
next_book_datetime=2023-11-07 13:26:57.878599
click_btn=None

now=2023-11-07 13:25:57.582323
next_book_datetime=2023-11-07 13:26:57.582301
master counter = 35, stride = 48.75, class_idx = 2
'35','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 13:26:28.881985'
next_book_datetime=2023-11-07 13:27:33.881964
click_btn=None

now=2023-11-07 13:26:33.918317
next_book_datetime=2023-11-07 13:27:43.918294
master counter = 36, stride = 48.75, class_idx = 3
'36','Hatha Basic 哈達基礎' by 'Una',FULL,'2023-11-07 13:26:49.480042'
next_book_datetime=2023-11-07 13:27:54.48

master counter = 65, stride = 48.75, class_idx = 2
'65','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 13:42:23.599294'
next_book_datetime=2023-11-07 13:43:28.599271
click_btn=None

now=2023-11-07 13:42:28.603665
next_book_datetime=2023-11-07 13:43:38.603639
master counter = 66, stride = 48.75, class_idx = 3
'66','Hatha Basic 哈達基礎' by 'Una',FULL,'2023-11-07 13:42:40.214789'
next_book_datetime=2023-11-07 13:43:45.214764
click_btn=None

now=2023-11-07 13:42:45.371726
next_book_datetime=2023-11-07 13:44:05.371695
master counter = 67, stride = 48.75, class_idx = 1
'67','Yoga Tone 2' by 'Una',FULL,'2023-11-07 13:43:02.079143'
next_book_datetime=2023-11-07 13:44:07.079121
click_btn=None

now=2023-11-07 13:43:06.785155
next_book_datetime=2023-11-07 13:44:06.785124
master counter = 68, stride = 48.75, class_idx = 2
'68','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 13:43:59.270489'
next_book_datetime=2023-11-07 13:45:04.270467
click_btn=None

now=2023-11-07 13:44:04.182447
next_book_datetime=2023-11-

now=2023-11-07 13:58:50.875337
next_book_datetime=2023-11-07 13:59:50.875308
master counter = 98, stride = 48.75, class_idx = 2
'98','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 13:59:55.911079'
next_book_datetime=2023-11-07 14:01:00.911056
click_btn=None

now=2023-11-07 14:00:01.866432
next_book_datetime=2023-11-07 14:01:11.866391
master counter = 99, stride = 48.75, class_idx = 3
'99','Yoga Tone 2' by 'Una',FULL,'2023-11-07 14:00:11.567500'
next_book_datetime=2023-11-07 14:01:16.567476
click_btn=None

now=2023-11-07 14:00:16.687943
next_book_datetime=2023-11-07 14:01:16.687919
master counter = 100, stride = 48.75, class_idx = 1
'100','Hatha Basic 哈達基礎' by 'Una',FULL,'2023-11-07 14:00:18.020644'
next_book_datetime=2023-11-07 14:01:23.020619
click_btn=None

now=2023-11-07 14:00:28.974749
next_book_datetime=2023-11-07 14:01:48.974721
master counter = 101, stride = 48.75, class_idx = 2
'101','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 14:01:32.572751'
next_book_datetime=2023-11-07 14:02:37.

now=2023-11-07 14:15:59.980356
next_book_datetime=2023-11-07 14:16:59.980335
master counter = 130, stride = 48.75, class_idx = 1
'130','Hatha Basic 哈達基礎' by 'Una',FULL,'2023-11-07 14:16:15.752169'
next_book_datetime=2023-11-07 14:17:20.752146
click_btn=None

now=2023-11-07 14:16:20.522217
next_book_datetime=2023-11-07 14:17:40.522187
master counter = 131, stride = 48.75, class_idx = 2
'131','Yoga Tone 2' by 'Una',FULL,'2023-11-07 14:17:20.640672'
next_book_datetime=2023-11-07 14:18:25.640650
click_btn=None

now=2023-11-07 14:17:25.610422
next_book_datetime=2023-11-07 14:18:25.610397
master counter = 132, stride = 48.75, class_idx = 3
'132','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 14:17:28.942153'
next_book_datetime=2023-11-07 14:18:33.942128
click_btn=None

now=2023-11-07 14:17:33.638017
next_book_datetime=2023-11-07 14:18:43.637994
master counter = 133, stride = 48.75, class_idx = 1
'133','Hatha Basic 哈達基礎' by 'Una',FULL,'2023-11-07 14:18:01.208631'
next_book_datetime=2023-11-07 14:

...

now=2023-11-07 20:04:59.529086
next_book_datetime=2023-11-07 20:06:09.529057
master counter = 739, stride = 32.5, class_idx = 1
'739','Yoga Tone 2' by 'Una',FULL,'2023-11-07 20:05:50.554219'
next_book_datetime=2023-11-07 20:06:55.554195
click_btn=None

now=2023-11-07 20:05:55.354740
next_book_datetime=2023-11-07 20:06:55.354716
master counter = 740, stride = 32.5, class_idx = 2
'740','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 20:06:30.200677'
next_book_datetime=2023-11-07 20:07:35.200653
click_btn=None

now=2023-11-07 20:06:35.336588
next_book_datetime=2023-11-07 20:07:45.336560
master counter = 741, stride = 32.5, class_idx = 1
'741','Yoga Tone 2' by 'Una',FULL,'2023-11-07 20:07:16.000811'
next_book_datetime=2023-11-07 20:08:21.000783
click_btn=None

now=2023-11-07 20:07:20.645651
next_book_datetime=2023-11-07 20:08:20.645619
master counter = 742, stride = 32.5, class_idx = 2
'742','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 20:08:06.035889'
next_book_datetime=2023-11-07 20:09:11.03586

master counter = 771, stride = 32.5, class_idx = 1
'771','Yoga Tone 2' by 'Una',FULL,'2023-11-07 20:30:03.524787'
next_book_datetime=2023-11-07 20:31:08.524763
click_btn=None

now=2023-11-07 20:30:08.640929
next_book_datetime=2023-11-07 20:31:08.640890
master counter = 772, stride = 32.5, class_idx = 2
'772','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 20:30:22.130385'
next_book_datetime=2023-11-07 20:31:27.130363
click_btn=None

now=2023-11-07 20:30:26.791933
next_book_datetime=2023-11-07 20:31:36.791900
master counter = 773, stride = 32.5, class_idx = 1
'773','Yoga Tone 2' by 'Una',FULL,'2023-11-07 20:31:29.320654'
next_book_datetime=2023-11-07 20:32:34.320630
click_btn=None

now=2023-11-07 20:31:34.038717
next_book_datetime=2023-11-07 20:32:34.038689
master counter = 774, stride = 32.5, class_idx = 2
'774','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 20:31:57.452843'
next_book_datetime=2023-11-07 20:33:02.452821
click_btn=None

now=2023-11-07 20:32:02.183247
next_book_datetime=2023-11-0

master counter = 803, stride = 32.5, class_idx = 1
'803','Yoga Tone 2' by 'Una',FULL,'2023-11-07 20:54:17.081350'
next_book_datetime=2023-11-07 20:55:22.081323
click_btn=None

now=2023-11-07 20:54:19.275149
next_book_datetime=2023-11-07 20:55:29.275125
now=2023-11-07 20:54:21.995816
next_book_datetime=2023-11-07 20:55:21.995795
master counter = 805, stride = 32.5, class_idx = 1
'805','Yoga Tone 2' by 'Una',FULL,'2023-11-07 20:55:42.656527'
next_book_datetime=2023-11-07 20:56:47.656504
click_btn=None

now=2023-11-07 20:55:47.607734
next_book_datetime=2023-11-07 20:56:47.607708
master counter = 806, stride = 32.5, class_idx = 2
'806','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 20:55:49.956669'
next_book_datetime=2023-11-07 20:56:54.956644
click_btn=None

now=2023-11-07 20:55:54.792594
next_book_datetime=2023-11-07 20:57:04.792569
master counter = 807, stride = 32.5, class_idx = 1
'807','Yoga Tone 2' by 'Una',FULL,'2023-11-07 20:57:08.258501'
next_book_datetime=2023-11-07 20:58:13.258477
c

master counter = 836, stride = 32.5, class_idx = 2
'836','Yoga Tone 2' by 'Una',FULL,'2023-11-07 21:18:30.430218'
next_book_datetime=2023-11-07 21:19:35.430194
click_btn=None

now=2023-11-07 21:18:35.051617
next_book_datetime=2023-11-07 21:19:35.051588
master counter = 837, stride = 32.5, class_idx = 1
'837','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 21:19:42.794241'
next_book_datetime=2023-11-07 21:20:47.794219
click_btn=None

now=2023-11-07 21:19:47.727181
next_book_datetime=2023-11-07 21:20:57.727154
master counter = 838, stride = 32.5, class_idx = 2
'838','Yoga Tone 2' by 'Una',FULL,'2023-11-07 21:19:55.735136'
next_book_datetime=2023-11-07 21:21:00.735113
click_btn=None

now=2023-11-07 21:20:00.375141
next_book_datetime=2023-11-07 21:21:00.375114
master counter = 839, stride = 32.5, class_idx = 1
'839','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 21:21:18.404576'
next_book_datetime=2023-11-07 21:22:23.404552
click_btn=None

master counter = 839, stride = 32.5, class_idx = 1
'839','Y

now=2023-11-07 21:42:48.524768
next_book_datetime=2023-11-07 21:43:48.524741
master counter = 869, stride = 32.5, class_idx = 1
'869','Yoga Wheel 瑜珈輪' by 'Una',FULL,'2023-11-07 21:43:35.128726'
next_book_datetime=2023-11-07 21:44:40.128701
click_btn=None

now=2023-11-07 21:43:39.812853
next_book_datetime=2023-11-07 21:44:49.812826
master counter = 870, stride = 32.5, class_idx = 2
'870','Yoga Tone 2' by 'Una',FULL,'2023-11-07 21:44:09.183937'
next_book_datetime=2023-11-07 21:45:14.183912
click_btn=None

now=2023-11-07 21:44:13.877717
next_book_datetime=2023-11-07 21:45:13.877690
now=2023-11-07 21:46:51.322114
next_book_datetime=2023-11-07 21:48:01.322087
master counter = 874, stride = 32.5, class_idx = 2
'874','Yoga Tone 2' by 'Una',FULL,'2023-11-07 21:46:59.859704'
next_book_datetime=2023-11-07 21:48:04.859680
click_btn=None

now=2023-11-07 21:47:04.390292
next_book_datetime=2023-11-07 21:48:04.390260
master counter = 875, stride = 32.5, class_idx = 1
'875','Yoga Wheel 瑜珈輪' by 'Una',F